TOTO予測
===

* [データ収集](001_データ収集.ipynb)で収集したデータを使う。
* [データ加工](002_データ加工.ipynb)で加工済みとする。

In [1]:
import download
import pandas as pd
df = download.get_db_data()
#display(df)

# 符号化

* チーム名・スタジアム名を符号化
* 勝ち・負け・引き分けを符号化
* 試合日を月のみ取得

In [2]:
home = pd.concat([df["ホーム"],df["アウェイ"]]).drop_duplicates().reset_index(drop=True).to_dict()
team_dict = {v:k for k, v in home.items()} # 辞書のキー・バリュー交換

In [3]:
tmp = [team_dict[df["ホーム"][i]] for i in range(len(df))]
df["home"] = tmp
tmp = [team_dict[df["アウェイ"][i]] for i in range(len(df))]
df["away"] = tmp
#display(df)

勝敗はホームチームの勝ち(WIN)、負け(LOSE)、引き分け(DRAW)の三種類に符号化する。  
最終出力を獲得勝ち点で出力することを考え、WIN=3/DRAW=1/LOSE=0としておく。

カップ戦とかでPK戦の結果も含まれているが、PKは引き分けととらえることにする。

In [4]:
import re
import numpy as np
wdl = []
# win-draw-lose VゴールとかPKとかはTOTO予想に関係ないので無視
LOSE = 0
DRAW = 1
WIN = 3
OTHER = np.nan
for result in df["スコア"]:
    tmp = re.split("[-()]",result)
    if len(tmp) < 2: # X-Xという形式でないものはスルー
        wdl.append(OTHER)
        continue
    if int(tmp[0]) > int(tmp[1]):
        wdl.append(WIN)
    elif int(tmp[0]) < int(tmp[1]):
        wdl.append(LOSE)
    else:
        wdl.append(DRAW)
df["result"] = wdl
#display(df)

対戦した時期として、月の情報を入れるものとする。

試合日から情報を検出する。

In [5]:
month = []
for match_day in df["試合日"]:
    tmp = match_day.split("/")[0]
    if tmp.isdigit() == True:
        month.append(int(tmp))
    else:
        month.append(np.nan)
df["month"] = month
#display(month)

In [6]:
df = df.rename(columns={"年度":"year"})
#display(df)

使うデータを1992年~2016年に絞る。  
2017年のデータは予測の正確性検証のために別にしておくため

In [7]:
data = df[df["year"] < 2017]

結果の異常値を弾く。

異常値としてNaN(np.nan)を使ったので、NaNを含む列をドロップする。

In [8]:
data = data.dropna()

ホームチームを限定して、入力データと2017年のデータを使った正解データを生成してみる。

In [9]:
def generate_trainng_data(df, year=2017):
    """
    ホームチームを指定し、学習用データ(指定した年度より前のデータから)を生成
    """
    data = df[df["year"] < year].dropna()
    x = data[["home","away","year","month"]]
    y = data["result"]
    return x, y

def generate_correct_data(df, year=2017):
    """
    ホームチームを指定し、モデルの確からしさを検証するためのデータを生成
    """
    data = df[df["year"] == year].dropna()
    x = data[["home","away","year","month"]]
    y = data[["result"]]
    return x, y

# 人工知能に入れてみる

とりあえず(うまくいくとは思えないが)ここまで加工したデータで、
* home
* away
* year
* month

を入力、resultを出力とするニューラルネットを構築してみたい。

In [10]:
import numpy as np
import tensorflow as tf
#import tensorboard as tb

In [ ]:
# 入力Xと出力yを定義
x, y = generate_trainng_data(df, 2017)
X = np.reshape(x.values,(-1,4,1))
y = np.reshape(y.values,(-1,1,1))
print(X.shape[1:],y.shape)
#
# ニューラルネットワークの定義
#
model = tf.keras.models.Sequential()	# Sequentialモデル
# Conv層を追加
model.add(tf.keras.layers.Conv1D(128,4,input_shape=X.shape[1:],activation="relu"))
model.add(tf.keras.layers.Conv1D(128,1,activation="relu"))
model.add(tf.keras.layers.Conv1D(128,1,activation="relu"))
model.add(tf.keras.layers.Conv1D(128,1,activation="relu"))
model.add(tf.keras.layers.MaxPool1D(1))
model.add(tf.keras.layers.Dropout(0.01))
model.add(tf.keras.layers.Conv1D(128,1,activation="relu"))
model.add(tf.keras.layers.Conv1D(128,1,activation="relu"))
model.add(tf.keras.layers.Conv1D(128,1,activation="relu"))
model.add(tf.keras.layers.Conv1D(128,1,activation="relu"))
model.add(tf.keras.layers.MaxPool1D(1))
model.add(tf.keras.layers.Dropout(0.01))
model.add(tf.keras.layers.Conv1D(128,1,activation="relu"))
model.add(tf.keras.layers.Conv1D(128,1,activation="relu"))
model.add(tf.keras.layers.Conv1D(128,1,activation="relu"))
model.add(tf.keras.layers.Conv1D(128,1,activation="relu"))
model.add(tf.keras.layers.MaxPool1D(1))
model.add(tf.keras.layers.Dropout(0.01))
model.add(tf.keras.layers.Conv1D(128,1,activation="relu"))
model.add(tf.keras.layers.Conv1D(128,1,activation="relu"))
model.add(tf.keras.layers.Conv1D(128,1,activation="relu"))
model.add(tf.keras.layers.Conv1D(128,1,activation="relu"))
model.add(tf.keras.layers.MaxPool1D(1))
model.add(tf.keras.layers.Dropout(0.01))
#model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(0.01))
model.add(tf.keras.layers.Dense(1,activation="softplus"))

# 学習の仕方を定義
# * 目的関数(ロス関数)としてmse
# * 最適化アルゴリズムにadam
# * 評価メトリクスとしてaccuracyを表示
model.compile(optimizer="adam", loss='mean_absolute_error',metrics=['accuracy'])

# Tensorboard用コールバック
#tb_cb = tf.keras.callbacks.TensorBoard(log_dir='./log')
#cbks = [tb_cb]
# 学習開始
# * バッチサイズ4
# * エポック数1000回
#history = model.fit(X, y, batch_size=len(X), epochs=1000, callbacks=cbks)
history = model.fit(X, y, batch_size=len(X), epochs=1000)

plot_history(history)
#tb.show_graph(tf.get_default_graph().as_graph_def()) # JupyterでTensorboardのGraphを表示させる

(4, 1) (14934, 1, 1)
Epoch 1/1000


In [30]:
# モデルから2017年の対戦データを予想し、predict列に表示
cx, cy = generate_correct_data(df,2017)
cxx = np.reshape(cx.values,(-1,4,1))
print(cxx.shape)

pred = model.predict(cxx)
predict = pd.DataFrame(np.reshape(pred,len(pred)))
display(pd.concat([
    cx.reset_index(drop=True), 
    cy.reset_index(drop=True), 
    predict],
    axis=1).rename(columns={0:"predict"}))

,away,year,month,result,predict
0,14,2017,3.0,3.0,7.736260
1,28,2017,3.0,3.0,7.767584
2,31,2017,4.0,3.0,7.769968
3,23,2017,4.0,3.0,7.759088
4,0,2017,5.0,0.0,7.691959
5,5,2017,5.0,1.0,7.709548
6,12,2017,6.0,0.0,7.730771
7,4,2017,7.0,3.0,7.708024
8,32,2017,7.0,3.0,7.769402
9,33,2017,8.0,1.0,7.769291


ある程度予想通りではあるが、こんな簡単には収束しない…。

# 統計学で試す

* スタジアム
* 何節か
* 対戦相手

In [16]:

p = len(df[(df["ホーム"]=="広島") & (df["result"]==3.0)])/len(df[(df["ホーム"]=="広島")])
print(p)

0.46579804560260585


In [21]:
df[(df["ホーム"] == "清水") & (df["アウェイ"]=="浦和")]

,year,大会,節,試合日,K/O時刻,ホーム,スコア,アウェイ,スタジアム,home,away,result,month
33,1992,ＹＮＣ,第７節第１日,10/03(土),16:00,清水,4-1,浦和,日本平,5,2,3.0,10.0
131,1993,サテライトリーグ,第５節第２日,06/20(日),14:00,清水,1-2,浦和,日本平,5,2,0.0,6.0
174,1993,Ｊ１ サントリー,第１７節第１日,07/10(土),18:59,清水,0-0(PK6-5),浦和,日本平,5,2,1.0,7.0
253,1993,Ｊ１ ＮＩＣＯＳ,第７節第１日,08/25(水),18:59,清水,3-0,浦和,日本平,5,2,3.0,8.0
307,1993,ＹＮＣ,第４節第１日,09/25(土),14:03,清水,3-2,浦和,札幌,5,2,3.0,9.0
543,1994,Ｊ１ サントリー,第１２節第１日,04/27(水),19:01,清水,2-1,浦和,草薙陸,5,2,3.0,4.0
762,1994,サテライトリーグ,第２０節第２日,09/11(日),13:01,清水,1-2,浦和,西ヶ谷,5,2,0.0,9.0
787,1994,Ｊ１ ＮＩＣＯＳ,第１２節第１日,09/21(水),19:02,清水,5-4,浦和,草薙陸,5,2,3.0,9.0
1134,1995,Ｊ１ サントリー,第２３節第１日,07/12(水),19:02,清水,2-3,浦和,日本平,5,2,0.0,7.0
1311,1995,Ｊ１ ＮＩＣＯＳ,第１４節第１日,10/04(水),19:03,清水,2-1,浦和,日本平,5,2,3.0,10.0
